In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm 
import itertools
import sklearn.feature_selection
import sklearn.model_selection as ms
import pickle
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import datetime
import xgboost as xgb
from scipy.stats import skew
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, RidgeCV, LassoCV, ElasticNetCV
from scipy import stats
from sklearn.preprocessing import normalize
from scipy.stats import skew
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
plt.style.use('ggplot')
pd.options.display.max_columns = None
pd.options.display.max_rows = 600

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
# Adding 'type' columns that contains 'train' or 'test', and add empty SalePrice columns to the test part
train.insert(0, 'type', 'train')
test.insert(0,'type','test')
test['SalePrice'] = 0.0

# merging the two dataframes
merged = pd.concat([train,test],axis=0).reset_index(drop=True)
merged['SalePrice'] = merged['SalePrice'].astype('float')

# Data cleaning
import Dprocessv4
train = Dprocessv4.dataProcess(merged);

In [4]:
# numerical and categorical features + EXCLUDED FIRST
excl_features = merged[['Id','type','SalePrice']].columns
cat_features = merged.select_dtypes(include = ["object"]).columns.drop('type')
num_features = merged.select_dtypes(exclude = ["object"]).columns.drop(['Id','SalePrice'])

mergedNum = merged[num_features]
mergedCat = merged[cat_features]
mergedExc = merged[excl_features]

mergedNAtotal = merged.isnull().sum().sort_values(ascending=False)

print("NAs for numerical features in merged : " + str(mergedNum.isnull().values.sum()))

if mergedNum.isnull().values.sum() != 0:
    mergedNum = mergedNum.fillna(mergedNum.median())
    print("Remaining NAs for numerical features in merged : " + str(mergedNum.isnull().values.sum()))

NAs for numerical features in merged : 0


In [5]:
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerics2 = []
for i in merged.columns:
    if merged[i].dtype in numeric_dtypes: 
        numerics2.append(i)

skew_features = merged[numerics2].apply(lambda x: skew(x)).sort_values(ascending=False)
skews = pd.DataFrame({'skew':skew_features})

In [6]:
high_skew = skew_features[skew_features > 0.5]
high_skew = high_skew
skew_index = high_skew.index

for i in skew_index:
    if i != 'SalePrice':
        merged[i]= boxcox1p(merged[i], boxcox_normmax(merged[i]+1))

        
skew_features2 = merged[numerics2].apply(lambda x: skew(x)).sort_values(ascending=False)
skews2 = pd.DataFrame({'skew':skew_features2})

In [7]:
# Create dummy features for categorical values via one-hot encoding

print("Remaining NAs for categorical features in merged : " + str(mergedCat.isnull().values.sum()))

print("NAs for categorical features in merged : " + str(mergedCat.isnull().values.sum()))

mergedCat = pd.get_dummies(mergedCat)


Remaining NAs for categorical features in merged : 0
NAs for categorical features in merged : 0


In [8]:
# rejoin excluded, categorical and numerical features
merged = pd.concat([mergedExc,mergedNum, mergedCat], axis = 1)

print("New number of features : " + str(merged.shape[1]))

New number of features : 335


In [9]:
merged.shape

(2915, 335)

In [10]:
train = merged[merged['type'] == 'train']
train.drop(columns=['Id','type'],inplace=True)
test = merged[merged['type'] == 'test']
test.drop(columns=['type','SalePrice'],inplace=True)

/Users/macpro/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [11]:
# taking the log of the saleprice

train['SalePrice'] = np.log(train['SalePrice'])
train['SalePrice'].describe()

/Users/macpro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


count    1456.000000
mean       12.022909
std         0.398649
min        10.460242
25%        11.774520
50%        12.001505
75%        12.273731
max        13.534473
Name: SalePrice, dtype: float64

### Linear Regression

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import sklearn.model_selection as ms
from scipy.stats import norm
from scipy.stats import skew
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.ensemble import (GradientBoostingRegressor , RandomForestRegressor,
                              BaggingRegressor,ExtraTreesRegressor,AdaBoostRegressor)
from scipy import stats
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, make_scorer
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

pd.set_option('display.max_columns', 85)
pd.set_option('display.max_rows', 85)

In [ ]:
y = train.SalePrice

## Defining the train/test split (80/20)

In [13]:
house_features = train.drop('SalePrice',axis = 1)
prices = train['SalePrice']

x_train, x_test, y_train, y_test = ms.train_test_split(house_features, prices, 
                                                       test_size=0.2, random_state=111)

## Ridge

In [16]:
# 981 
# 0.12077
# R2: 0.9420
# trainRMSE: 0.096945
# testRMSE: 0.101837

ridcv = RidgeCV(alphas=np.linspace(1e-5,20,10000), normalize=False, scoring=None)

ridcv.fit(x_train,y_train)

trainRMSE = np.sqrt(mean_squared_error(y_true=y_train, y_pred=ridcv.predict(x_train)))
testRMSE = np.sqrt(mean_squared_error(y_true=y_test, y_pred=ridcv.predict(x_test)))

print('The best alpha is: {:.6f}\n\nThe ridge intercept is: {:.2f}\n\n{}\n\nThe model score is {:.4f}\n\nTrain RMSE: {:.6f}\n\nTest RMSE: {:.6f}\n\nDiff in RMSE: {:.3f}\n\n'.format(
    ridcv.alpha_,ridcv.intercept_, pd.Series(ridcv.coef_, index=x_train.columns), ridcv.score(x_train, y_train), trainRMSE,testRMSE,(testRMSE - trainRMSE)))

# pd.DataFrame({'Id': test['Id'], 'SalePrice': np.exp(ridcv.predict(test.drop(columns='Id')))}
#             ).to_csv('ridcv {}.csv'.format(str(datetime.datetime.now())[:19].replace(':','.')
#                                            ), index=False)

The best alpha is: 20.244524

The ridge intercept is: 5.31

LotFrontage              0.000490
LotArea                  0.000002
OverallQual              0.050000
OverallCond              0.040034
YearBuilt                0.002033
YearRemodAdd             0.000680
MasVnrArea               0.000026
BsmtFinSF1               0.000058
BsmtFinSF2               0.000044
BsmtUnfSF               -0.000011
TotalBsmtSF              0.000091
1stFlrSF                 0.000106
2ndFlrSF                 0.000094
LowQualFinSF            -0.000033
GrLivArea                0.000166
BsmtFullBath             0.020115
BsmtHalfBath             0.005169
FullBath                 0.020433
HalfBath                 0.028653
BedroomAbvGr            -0.004380
KitchenAbvGr            -0.013572
TotRmsAbvGrd            -0.000793
Fireplaces               0.003851
GarageYrBlt             -0.000037
GarageCars               0.023184
GarageArea               0.000080
WoodDeckSF               0.000088
OpenPorchSF           

## Elastic-net

In [ ]:
# 917
# 0.11994
# R2: 0.9359
# trainRMSE: 0.101935
# testRMSE: 0.100320
# diff: -0.002

enet = ElasticNetCV(l1_ratio=np.linspace(0.01,1,100), alphas=np.linspace(1e-5,20,1000), max_iter=1000, normalize=False, cv=10, n_jobs=-1)

enet.fit(x_train, y_train)

trainRMSE = np.sqrt(mean_squared_error(y_true=y_train, y_pred=enet.predict(x_train)))
testRMSE = np.sqrt(mean_squared_error(y_true=y_test, y_pred=enet.predict(x_test)))

print('The best alpha is: {:.6f}\n\nThe elastic-net intercept is: {:.2f}\n\n{}\n\nThe model score is {:.4f}\n\nTrain RMSE: {:.6f}\n\nTest RMSE: {:.6f}\n\nDiff in RMSE: {:.3f}\n\n'.format(
    enet.alpha_,enet.intercept_, pd.Series(enet.coef_, index=x_train.columns), enet.score(x_train, y_train), trainRMSE,testRMSE,(testRMSE - trainRMSE)))

# pd.DataFrame({'Id': test['Id'], 'SalePrice': np.exp(enet.predict(test.drop(columns='Id')))}
#             ).to_csv('enet {}.csv'.format(str(datetime.datetime.now())[:19].replace(':','.')
#                                            ), index=False)

## Lasso

In [ ]:
#  alpha           UJ range              np.linspace(1e-4,1e-2,1000)
#  CV          100         10          100
# R2:        0.9366      0.9399      0.9374
# trainRMSE: 0.101363    0.098698    0.100718
# testRMSE:  0.100012    0.100196    0.099977
# diff:b       0.00       0.00         0.00

lassocv = LassoCV(alphas=np.linspace(1e-4,1e-3,1000), normalize=False,
                      max_iter=10000, n_jobs=-1, cv=100)

lassocv.fit(x_train, y_train)

trainRMSE = np.sqrt(mean_squared_error(y_true=y_train, y_pred=lassocv.predict(x_train)))
testRMSE = np.sqrt(mean_squared_error(y_true=y_test, y_pred=lassocv.predict(x_test)))

print('The best alpha is: {:.6f}\n\nThe lasso intercept is: {:.2f}\n\n{}\n\nThe model score is {:.4f}\n\nTrain RMSE: {:.6f}\n\nTest RMSE: {:.6f}\n\nDiff in RMSE: {:.3f}\n\n'.format(
    lassocv.alpha_,lassocv.intercept_, pd.Series(lassocv.coef_, index=x_train.columns), lassocv.score(x_train, y_train), trainRMSE,testRMSE,(testRMSE - trainRMSE)))

# pd.DataFrame({'Id':test['Id'],'SalePrice': np.exp(lassocv.predict(test.drop(columns='Id')))}
#             ).to_csv('lasso {}.csv'.format(str(datetime.datetime.now())[:19].replace(':','.')
#                                            ), index=False)

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=1000,max_depth=7,max_features='auto', min_samples_split=2, min_samples_leaf=1,n_jobs=-1)
rf.fit(x_train, y_train)

scores = ms.cross_val_score(rf, x_train, y_train, cv=10)

print(scores.mean())

trainRMSE = np.sqrt(mean_squared_error(y_true=y_train, y_pred=rf.predict(x_train)))
testRMSE = np.sqrt(mean_squared_error(y_true=y_test, y_pred=rf.predict(x_test)))

print('\nThe model score is {:.4f}\n\nTrain RMSE: {:.6f}\n\nTest RMSE: {:.6f}\n\nDiff in RMSE: {:.3f}\n\n'.format(
    rf.score(x_train, y_train), trainRMSE,testRMSE,(testRMSE - trainRMSE)))


# pd.DataFrame({'Id':test['Id'],'SalePrice': np.exp(rf.predict(test.drop(columns='Id')))}
#             ).to_csv('rf {}.csv'.format(str(datetime.datetime.now())[:19].replace(':','.')
#                                            ), index=False)

# 1000 trees
# 0.8395225412896815
# R2: 0.9517
# Train RMSE: 0.088419
# Test RMSE: 0.129310
# Diff in RMSE: 0.041

## Output csv

In [ ]:
# pd.DataFrame({'Id': test['Id'], 'SalePrice': np.exp(ridcv.predict(test.drop(columns='Id')))}
#             ).to_csv('ridcv {}.csv'.format(str(datetime.datetime.now())[:19].replace(':','.')
#                                            ), index=False)
# pd.DataFrame({'Id': test['Id'], 'SalePrice': np.exp(enet.predict(test.drop(columns='Id')))}
#             ).to_csv('enet {}.csv'.format(str(datetime.datetime.now())[:19].replace(':','.')
#                                            ), index=False)
# pd.DataFrame({'Id':test['Id'],'SalePrice': np.exp(lassocv.predict(test.drop(columns='Id')))}
#             ).to_csv('lasso {}.csv'.format(str(datetime.datetime.now())[:19].replace(':','.')
#                                            ), index=False)
# pd.DataFrame({'Id':test['Id'],'SalePrice': np.exp(rf.predict(test.drop(columns='Id')))}
#             ).to_csv('rf {}.csv'.format(str(datetime.datetime.now())[:19].replace(':','.')
#                                            ), index=False)